# **MÓDULO 32 - Exercício**
# Random Forest


Nesta tarefa, vocês vão trabalhar com uma base de dados de avaliações de vinhos, onde o objetivo é prever a pontuação dos vinhos usando o algoritmo de Random Forest para classificação multiclasse.

In [40]:
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go

In [30]:
df = pd.read_csv("C:/Users/yurid/Downloads/winequality-red.csv", delimiter=',')

df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


**Vamos conhecer nossa base:**

Características dos Vinhos (Features)

Fixed Acidity: Acidez fixa do vinho.

Volatile Acidity: Acidez volátil do vinho.

Citric Acid: Quantidade de ácido cítrico no vinho.

Residual Sugar: Açúcar residual presente no vinho.

Chlorides: Nível de cloretos no vinho.

Free Sulfur Dioxide: Dióxido de enxofre livre no vinho.

Total Sulfur Dioxide: Quantidade total de dióxido de enxofre no vinho.

Density: Densidade do vinho.

pH: Nível de pH do vinho.

Sulphates: Quantidade de sulfatos no vinho.

Alcohol: Teor alcoólico do vinho.



**Variável de Saída (Target):**

Quality: Pontuação do vinho baseada em dados sensoriais, variando de 0 a 10.


Esta abordagem permitirá que vocês explorem como diferentes características químicas influenciam a qualidade dos vinhos e como o Random Forest pode ser usado para fazer previsões precisas com base nesses dados.

# 1 - Realize a primeira etapa de pré processamento dos dados.

A) Verifique os tipos de dados.


B) Verifique os dados faltantes, se houver dados faltantes faça a substituição ou remoção justificando sua escolha.

In [3]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
None
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides        

# 2 - Realize a segunda e terceita etapa de pré processamento dos dados.

A) Utilize a função describe para identificarmos outliers e verificarmos a distribuição dos dados.

B) Verifique o balanceamento da váriavel Target.

C)  Plote o gráfico ou a tabela e indique as variáveis que te parecem mais "fortes" na correlação para nosso modelo.

D) Crie um novo dataframe apenas com as váriaveis que parecem ter maior correlação com a target. (Negativa ou positiva)


In [4]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [23]:
colors = ['orchid', 'tomato', 'olive', 'yellow', 'turquoise', 'pink', 'blue', 'orange', 'brown', 'green', 'purple', 'magenta']
fig = make_subplots(cols=12, rows =2)

for i, coluna in enumerate(df.columns):
    fig.add_traces(go.Histogram(x = df[coluna],
                          name = coluna,
                                marker_color = colors[i]),
                   rows=1,
                   cols=i+1)
    fig.add_traces(go.Box(y = df[coluna],
                          name = coluna,
                          marker_color = colors[i],
                          showlegend = False),
                   rows=2,
                   cols=i+1)

fig.update_layout(title_text = 'Distribuição das variáveis')
fig.show()

Conforme os histogramas e boxplots, as distribuições não apresentam outliers significativos. Isso é reforçado pela média próxima à mediana em todos os casos. Agora, faremos a verificação de balanceamento e a matriz de correlação.


In [27]:
print(df['quality'].value_counts())

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64


In [91]:
corr = df.corr()

fig = go.Figure()

fig.add_traces(go.Heatmap(z = corr,
                          y= corr.index,
                          x = corr.index,
                          colorscale='bluered',
                          texttemplate='%{z}'))

Conforme a matriz de correlação acima, as variáveis que mais influenciam a variável target são:
   - Positivamente: alcohol, sulphates e citric acid;
   - Negativamente: volatile acidity
Ressalta-se que o critério de escolha, já que as correlações foram inferiores a 0.5, foi o intervalo 0.2<x<0.5, de forma a abranger um número mais significativo de variáveis.
Assim, criamos o novo data frame:

In [92]:
df_modelo = df.drop(['pH', 'free sulfur dioxide', 'chlorides', 'residual sugar', 'fixed acidity', 'density', 'total sulfur dioxide'], axis = 1)
df_modelo

,volatile acidity,citric acid,sulphates,alcohol,quality
0,0.700,0.00,0.56,9.4,5
1,0.880,0.00,0.68,9.8,5
2,0.760,0.04,0.65,9.8,5
3,0.280,0.56,0.58,9.8,6
4,0.700,0.00,0.56,9.4,5
...,...,...,...,...,...
1594,0.600,0.08,0.58,10.5,5
1595,0.550,0.10,0.76,11.2,6
1596,0.510,0.13,0.75,11.0,6
1597,0.645,0.12,0.71,10.2,5


# 3 - Preparação Final dos Dados

A) Separe a base em X(Features) e Y(Target)

B) Separe a base em treino e teste.


In [123]:
X = df_modelo.drop('quality', axis = 1)
Y = df_modelo['quality']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

dataset = {'x_train': x_train, 'y_train': y_train, 'x_test': x_test, 'y_test': y_test}

for nome, variavel in dataset.items():
    print(f'Tamanho de {nome}: {variavel.shape}')



Tamanho de x_train: (1279, 4)
Tamanho de y_train: (1279,)
Tamanho de x_test: (320, 4)
Tamanho de y_test: (320,)


# 4 - Modelagem

A) Inicie e treine o modelo de Random Forest

B) Aplique a base de teste o modelo.


In [101]:
modelo = RandomForestClassifier()
modelo.fit(x_train, y_train)
y_pred = modelo.predict(x_test)


# 5 - Avaliação

A) Avalie as principais métricas da Claissificação e traga insights acerca do resultado, interprete os valores achados.

B) Você nota que o modelo teve dificuldade para prever alguma classe? Se sim, acredita que tenha relação com o balanceamento dos dados? Explique.


In [102]:
relatorio = classification_report(y_test, y_pred)
print(relatorio)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        10
           5       0.68      0.73      0.70       130
           6       0.60      0.63      0.61       132
           7       0.55      0.52      0.54        42
           8       0.00      0.00      0.00         5

    accuracy                           0.62       320
   macro avg       0.30      0.31      0.31       320
weighted avg       0.59      0.62      0.61       320



C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Nota-se que o modelo apresentou dificuldades para prever as classes 3, 4 e 8 por causa do balanceamento dos dados, já que há muito menos dados dessas classes. Para confirmar, vamos testar o balanceamento dos dados e comparar as matrizes de confusão.

In [105]:
#Balanceando os dados de treino
sm = SMOTE(random_state=42)
x_train_bal, y_train_bal = sm.fit_resample(x_train, y_train)

#Treinando o modelo
modelo_bal = RandomForestClassifier()
modelo_bal.fit(x_train_bal, y_train_bal)

#Fazendo as previsões
y_pred_bal = modelo_bal.predict(x_test)

#Avaliando o modelo
relatorio_bal = classification_report(y_test, y_pred_bal)
print(relatorio_bal)



              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.08      0.20      0.12        10
           5       0.72      0.65      0.68       130
           6       0.63      0.55      0.59       132
           7       0.57      0.69      0.62        42
           8       0.11      0.20      0.14         5

    accuracy                           0.59       320
   macro avg       0.35      0.38      0.36       320
weighted avg       0.64      0.59      0.61       320



In [122]:
matriz = confusion_matrix(y_test, y_pred)
matriz_bal = confusion_matrix(y_test, y_pred_bal)
classes = ['3', '4', '5', '6', '7', '8']
fig = make_subplots(rows=1, cols = 2, subplot_titles=['Dados desbalanceados', 'Dados balanceados'])

fig.add_traces(go.Heatmap(z = matriz,
                          y=classes,
                          x=classes,
                          texttemplate='%{z}',
                          colorscale = 'burg'),
               rows=1,
               cols=1)
fig.add_traces(go.Heatmap(z = matriz_bal,
                          x=classes,
                          y=classes,
                          texttemplate='%{z}',
                          colorscale = 'burg',
                          showscale=False),
               rows=1,
               cols=2)

Diante dos resultados, podemos inferir melhoria na precisão para todas as classes, exceto a classe 3 ainda em virtude do baixo número de amostras. Todavia, o recall aumentou somente para as classes 4, 7 e 8, enquanto caiu para as classes 5 e 6. No que tange à acurácia geral do modelo, temos uma queda de 3%. Conforme as matrizes de confusão, o modelo performou melhor sem o balanceamento dos dados, de modo que houve mais erros nos dados balanceados, o que confirma o exposto pelas métricas analisadas. Aqui, é necessário estabelecer que, embora o modelo tenha sido capaz de prever mais classes com os dados balanceados, os erros foram mais presentes. Então, é preferível optar pelo modelo desbalanceado, que possui maior acurácia e recall.

# 5 - Melhorando os Hyperparametros

A) Defina o Grid de parametros que você quer testar

B) Inicie e Treine um novo modelo utilizando o random search.

C) Avalie os resultados do modelo.

D) Você identificou melhorias no modelo após aplicar o random search? Justifique.


ps. Essa parte da atividade demorará um pouco para rodar!

In [126]:
#Definindo os melhores parâmetros com Random Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(estimator=modelo, param_distributions=param_grid,
                                   n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42, scoring='accuracy')
random_search.fit(x_train, y_train)

best_params = random_search.best_params_
print(best_params)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}


In [127]:
#Treinando o modelo melhorado
improved_model = random_search.best_estimator_
improved_model.fit(x_train, y_train)
y_pred_improved = improved_model.predict(x_test)
relatorio = classification_report(y_test, y_pred_improved)
print(relatorio)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       1.00      0.10      0.18        10
           5       0.68      0.74      0.71       130
           6       0.59      0.61      0.60       132
           7       0.56      0.55      0.55        42
           8       0.00      0.00      0.00         5

    accuracy                           0.62       320
   macro avg       0.47      0.33      0.34       320
weighted avg       0.62      0.62      0.61       320



C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yurid\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [129]:
improved_matrix = confusion_matrix(y_test, y_pred_improved)
fig = go.Figure()

fig.add_traces(go.Heatmap(z = improved_matrix,
                          x= classes,
                          y = classes,
                          texttemplate='%{z}',
                          colorscale = 'burg'))

fig.update_layout(title_text = 'Matriz de confusão do modelo melhorado')

Conforme o relatório e a matriz de confusão, as métricas acurácia, precisão e recall melhoraram para todas as classes, de modo que o melhoramento do modelo foi um sucesso.

# 6 - Chegando a perfeição

Baseado em tudo que você já aprendeu até agora, quais outras técnicas você acredita que poderiam ser aplicadas ao modelo para melhorar ainda mais suas previsões?

Por se tratar de um modelo de Random Forest, que tem melhor desempenho sem padronização dos dados, uma saída seria um n amostral maior para as classes minoritárias, de modo a produzir um modelo capaz de prever um maior número dela. Além disso, o tratamento de outliers (remoção das classes minoritárias por serem bem menos representativas que as demais) poderia melhorar bastante o modelo. Além disso, optar pelo grid search ao invés do random search para os hiperparâmetros pode ajudar a defini-los melhor para o modelo pela busca exaustiva. Por fim, uma técnica de validação cruzada também poderia ser útil.